In [5]:
""" Datascience, 2022 2nd semester, Konkuk University
This source code runs on docker images `tensorflow/tensorflow:2.9.1-gpu-jupyter` and `python:3.7.14-buster`
* Writer: WONJUN PARK

No. 1
"""
import pandas as pd

def sortByStudentNumber(e):
    return e['sno']

result = []

# Read data from excel
excel_file = 'score.xlsx'
data = pd.read_excel(excel_file)

# Filter data, according to the problem conditions
for i in range(len(data)):
    midterm_score = data['midterm'][i]
    final_score = data['final'][i]
    
    if midterm_score >= 20:
        if final_score >= 20:
            result.append(data.iloc[i][['sno', 'midterm', 'final']])

# Sort data, according to the problem conditions
result.sort(key=sortByStudentNumber)

# Print data
result = pd.DataFrame(result)
print(result)

    sno  midterm  final
0     1    35.00  30.10
1     2    34.12  30.10
2     3    34.30  29.75
3     4    31.50  29.05
4     5    33.25  21.35
5     6    24.50  32.20
6     7    23.80  28.70
7     8    28.87  25.90
8     9    29.75  21.70
10   11    29.57  21.35
11   12    31.50  20.65
12   13    29.75  21.00
14   15    28.00  23.10
15   16    24.50  24.85
18   19    24.50  24.50
19   20    28.00  20.65
26   27    22.75  22.05
28   29    26.25  22.40
36   37    24.50  20.30


In [6]:
import os
from dotenv import load_dotenv

load_dotenv()

HOST = os.getenv('HOST')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
DATABASE = os.getenv('DATABASE')

In [7]:
"""
No. 2
"""
import pymysql

connection = pymysql.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    db=DATABASE,
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        sql = """
            create table if not exists score(
                student_number	int		primary key,
                attendance	decimal(3,2),
                homework	decimal(4,2),
                discussion	int,
                midterm		decimal(4,2),
                final		decimal(4,2),
                score		decimal(4,2),
                grade 		char(1)
            );
            """
        cursor.execute(sql)

        sql = """
            insert IGNORE into score(student_number, attendance, homework, discussion, midterm, final, score, grade)
            values(%s, %s, %s, %s, %s, %s, %s, %s);
        """
        cursor.executemany(sql, list([(x) for x in data.values.tolist()]))
        connection.commit()
        
        cursor.close()

In [8]:
"""
No. 3
"""

result = pd.DataFrame()

connection = pymysql.connect(
    host=HOST,
    user=USER,
    password=PASSWORD,
    db=DATABASE,
    cursorclass=pymysql.cursors.DictCursor
)

with connection:
    with connection.cursor() as cursor:
        sql = "select student_number, midterm, final from score where midterm >= 20 and final >= 20"
        row = cursor.execute(sql)

        row = cursor.fetchone()
        while row:
            temp = pd.DataFrame([[row['student_number'], row['midterm'], row['final']]],
                                    columns=['student_number', 'midterm', 'final'])
            result = pd.concat([result, temp], ignore_index=True)
            row = cursor.fetchone()
        
        cursor.close()

result.sort_values(by=['student_number'], inplace=True)
print(result)

    student_number midterm  final
0                1   35.00  30.10
1                2   34.12  30.10
2                3   34.30  29.75
3                4   31.50  29.05
4                5   33.25  21.35
5                6   24.50  32.20
6                7   23.80  28.70
7                8   28.87  25.90
8                9   29.75  21.70
9               11   29.57  21.35
10              12   31.50  20.65
11              13   29.75  21.00
12              15   28.00  23.10
13              16   24.50  24.85
14              19   24.50  24.50
15              20   28.00  20.65
16              27   22.75  22.05
17              29   26.25  22.40
18              37   24.50  20.30
